In [3]:
df = spark.read.csv('file:/home/hduser/Titanic.csv',inferSchema=True, header=True)
#df = spark.read.format("csv").option("inferSchema",True).option("header",True).load('file:/home/hduser/Titanic.csv')
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|        892|       0|     3|    Kelly, Mr. James|  male|34.5|    0|    0|          330911| 7.8292| null|       Q|
|        893|       1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|    7.0| null|       S|
|        894|       0|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|          240276| 9.6875| null|       Q|
|        895|       0|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|          315154| 8.6625| null|       S|
|        896|       1|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|         3101298|12.2875| null|       S|
|        897|       0|     3|Svensson, Mr. Joh...|  male|14.0|    0|    0|      

In [6]:
df.printSchema()
df.describe().show()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+-------+------------------+-------------------+------------------+--------------------+------+------------------+------------------+------------------+------------------+------------------+-----+--------+
|summary|       PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|             Parch|            Ticket|              Fare|Cabin|Embarked|
+-------+------------------+-------------------+------------------+--------------------+------+------------------+-----------

In [11]:
#Columns used for features and label
rm_columns = df.select(['Survived','Pclass', 
                       'Sex','Age','SibSp', 
                       'Parch','Fare','Embarked'])

#Remove null records from the dataset
result = rm_columns.na.drop()

result.show()


+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|34.5|    0|    0| 7.8292|       Q|
|       1|     3|female|47.0|    1|    0|    7.0|       S|
|       0|     2|  male|62.0|    0|    0| 9.6875|       Q|
|       0|     3|  male|27.0|    0|    0| 8.6625|       S|
|       1|     3|female|22.0|    1|    1|12.2875|       S|
|       0|     3|  male|14.0|    0|    0|  9.225|       S|
|       1|     3|female|30.0|    0|    0| 7.6292|       Q|
|       0|     2|  male|26.0|    1|    1|   29.0|       S|
|       1|     3|female|18.0|    0|    0| 7.2292|       C|
|       0|     3|  male|21.0|    2|    0|  24.15|       S|
|       0|     1|  male|46.0|    0|    0|   26.0|       S|
|       1|     1|female|23.0|    1|    0|82.2667|       S|
|       0|     2|  male|63.0|    1|    0|   26.0|       S|
|       1|     1|female|47.0|    1|    0| 61.175|       

In [13]:
# Importing the required libraries 
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

# Converting the Sex Column 
sexIdx = StringIndexer(inputCol='Sex', outputCol='SexIndex') 
sexEncode = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

# Converting the Embarked Column 
embarkIdx = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIndex') 
embarkEncode = OneHotEncoder(inputCol='EmbarkedIndex', outputCol='EmbarkedVec')

# Vectorizing the data into a new column "features"  
# which will be our input/features class 
assembler = VectorAssembler(inputCols=['Pclass', 
                                       'SexVec','Age', 
                                       'SibSp','Parch', 
                                       'Fare','EmbarkedVec'], 
                                    outputCol='features')



# Importing Model 
from pyspark.ml.classification import LogisticRegression 

log_reg = LogisticRegression(featuresCol='features',labelCol='Survived')


In [24]:
# Importing Pipeline and Model 
from pyspark.ml import Pipeline 

# Creating the pipeline 
pipeline = Pipeline(stages=[sexIdx, sexEncode,embarkIdx, embarkEncode, assembler, log_reg])

# Splitting the data into train and test 
train_data, test_data = result.randomSplit([0.7, .3])

# Fitting the model on training data 
fit_model = pipeline.fit(train_data)

# Storing the results on test data 
results = fit_model.transform(test_data) 

# Showing the results 
results.select("Survived","features","rawPrediction","prediction").show(truncate=False)

results.printSchema()


+--------+---------------------------------------+----------------------------------------+----------+
|Survived|features                               |rawPrediction                           |prediction|
+--------+---------------------------------------+----------------------------------------+----------+
|0       |[1.0,1.0,13.0,2.0,2.0,262.375,0.0,1.0] |[18.44673367200406,-18.44673367200406]  |0.0       |
|0       |[1.0,1.0,23.0,0.0,0.0,93.5,1.0,0.0]    |[23.557386055683256,-23.557386055683256]|0.0       |
|0       |[1.0,1.0,24.0,1.0,0.0,60.0,1.0,0.0]    |[24.809846315919824,-24.809846315919824]|0.0       |
|0       |[1.0,1.0,25.0,0.0,0.0,26.0,0.0,1.0]    |[26.769583202904922,-26.769583202904922]|0.0       |
|0       |[1.0,1.0,27.0,1.0,0.0,136.7792,0.0,1.0]|[21.973125989194433,-21.973125989194433]|0.0       |
|0       |[1.0,1.0,30.0,0.0,0.0,26.0,1.0,0.0]    |[25.636592890704012,-25.636592890704012]|0.0       |
|0       |[1.0,1.0,30.0,0.0,0.0,45.5,1.0,0.0]    |[24.83264075270478,-24.

In [27]:
# Importing the evaluator 
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Calling the evaluator 
res = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

# Evaluating the AUC on results 
ROC_AUC = res.evaluate(results)

print(ROC_AUC)
#In general, an AUC value above 0.7 is considered good


1.0


In [29]:
fit_model.write().overwrite().save("file:/home/hduser/titanic-predictions")
print("Model Saved")

Model Saved


In [34]:
df = spark.read.csv('file:/home/hduser/Titanic_realdata.csv',inferSchema=True, header=True)
resultdf = df.select(['Survived','Pclass', 
                       'Sex','Age','SibSp', 
                       'Parch','Fare','Embarked']).na.drop()


In [35]:
from pyspark.ml import PipelineModel

plmodel = PipelineModel.load('file:/home/hduser/titanic-predictions')

result = plmodel.transform(resultdf)

result.show()

+--------+------+----+---+-----+-----+-------+--------+--------+-------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass| Sex|Age|SibSp|Parch|   Fare|Embarked|SexIndex|       SexVec|EmbarkedIndex|  EmbarkedVec|            features|       rawPrediction|         probability|prediction|
+--------+------+----+---+-----+-----+-------+--------+--------+-------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|       1|     3|male| 32|    0|    0|   7.85|       S|     0.0|(1,[0],[1.0])|          0.0|(2,[0],[1.0])|[3.0,1.0,32.0,0.0...|[18.8798774840341...|[0.99999999368209...|       0.0|
|       1|     2|male| 30|    0|    0|   13.0|       S|     0.0|(1,[0],[1.0])|          0.0|(2,[0],[1.0])|[2.0,1.0,30.0,0.0...|[22.5205855245381...|[0.99999999983425...|       0.0|
|       1|     1|male| 45|    1|    0|52.5542|       S|     0.0|(1,[0],[1.0])|          0.0|(2,